In [25]:
from mceval.utils import LabelledDataset, GenerationPipeline, PromptFormatter, LlamaGenerationPipeline
import datasets
from transformers import AutoTokenizer 

In [26]:
# %ls

# %cd ..

In [27]:
import pandas as pd

# results_path = 'mceval_run1'
models = {'llama2-chat': 'meta-llama/Llama-2-7b-chat-hf', 'llama3-instruct': 'meta-llama/Meta-Llama-3-8B-Instruct'}
datasets = ['harmless', 'dilemmas']
prompt_var = ['standard', 'newline']
runs = ['run3', 'run4']


In [28]:
# # decode function
# def decode_top_0(example: dict, tokenizer) -> dict:
#     # takes


In [29]:
comparison_cols = [
    'is_A_standard_with_reasoning', 'is_A_standard_no_reasoning',
    'is_A_newline_with_reasoning', 'is_A_newline_no_reasoning',
    'is_other_standard_with_reasoning', 'is_other_standard_no_reasoning',
    'is_other_newline_with_reasoning', 'is_other_newline_no_reasoning'
]

In [30]:
results_cols = [
                    'model', 'dataset', 'run', 
                    'num_A_standard_prompt_with_reasoning', 'num_A_newline_prompt_with_reasoning', 
                    'num_A_standard_prompt_no_reasoning', 'num_A_newline_prompt_no_reasoning',
                    'num_other_standard_prompt_with_reasoning', 'num_other_newline_prompt_with_reasoning',
                    'num_other_standard_prompt_no_reasoning', 'num_other_newline_prompt_no_reasoning',
                    'num_prompt_var_cause_flip_with_reasoning', 
                    'num_prompt_var_cause_flip_no_reasoning', 
                    'num_reasoning_cause_flip_standard_prompt',
                    'num_reasoning_cause_flip_newline_prompt',
                    'num_reasoning_changed_prompt_var_flip',
                    ]

In [31]:
# TODO change so that other is not assumed to be small

# init results full with empty columns
results_full = pd.DataFrame(
    columns=results_cols
    )
for model_identifier in models.keys():
    tokenizer = AutoTokenizer.from_pretrained(models[model_identifier])
    # get token for 'A' and 'B'
    token_A = tokenizer(' A', return_tensors='pt')['input_ids'][0][-1].item()
    token_B = tokenizer(' B', return_tensors='pt')['input_ids'][0][-1].item()
    print(f"token A: {token_A}, token B: {token_B}")
    if not isinstance(token_A, int):
        print('Token A is type:', type(token_A))
        raise ValueError('Token A is not an integer')
    if not isinstance(token_B, int):
        print('Token B is type:', type(token_B))
        raise ValueError('Token B is not an integer')
        
    for dataset in datasets:
        for run in runs:
            # if run == 'run1' and model_identifier == 'llama2-chat':
            #     # special processing for run1 llama2-chat
            #     results_path = f'old_results/{model_identifier}/{dataset}_decoded_answers_{prompt}.jsonl'
            # else:
            # init results df
            
            # TODO Maybe restructure the columns, a bit messy
            results_temp = {}
            
            results_temp['model'] = model_identifier
            results_temp['dataset'] = dataset
            results_temp['run'] = run
            
            # load dataset
            
            standard_with_reasoning_path = f'mceval_{run}/{model_identifier}/scores_with_reasoning_{dataset}_standard.jsonl'
            standard_no_reasoning_path = f'mceval_{run}/{model_identifier}/scores_without_reasoning_{dataset}_standard.jsonl'
            newline_with_reasoning_path = f'mceval_{run}/{model_identifier}/scores_with_reasoning_{dataset}_newline.jsonl'
            newline_no_reasoning_path = f'mceval_{run}/{model_identifier}/scores_without_reasoning_{dataset}_newline.jsonl'
            
            # load as df
            # print(f"loading {standard_with_reasoning_path}..")
            standard_with_reasoning = pd.read_json(standard_with_reasoning_path, lines=True, orient='records')
            # print(f"loaded {standard_with_reasoning_path}")
            standard_no_reasoning = pd.read_json(standard_no_reasoning_path, lines=True, orient='records')
            # print(f"loaded {standard_no_reasoning_path}")
            newline_with_reasoning = pd.read_json(newline_with_reasoning_path, lines=True, orient='records')
            # print(f"loaded {newline_with_reasoning_path}")
            newline_no_reasoning = pd.read_json(newline_no_reasoning_path, lines=True, orient='records')
            # print(f"loaded {newline_no_reasoning_path}")
            
            id_colname = 'top_k_ids'
            # top_id_is_A and other
            comparison_df = pd.DataFrame(columns=comparison_cols)
            comparison_df['is_A_standard_with_reasoning'] = standard_with_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            comparison_df['is_A_standard_no_reasoning'] = standard_no_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            comparison_df['is_A_newline_with_reasoning'] = newline_with_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            comparison_df['is_A_newline_no_reasoning'] = newline_no_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            comparison_df['is_other_standard_with_reasoning'] = standard_with_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            comparison_df['is_other_standard_no_reasoning'] = standard_no_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            comparison_df['is_other_newline_with_reasoning'] = newline_with_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            comparison_df['is_other_newline_no_reasoning'] = newline_no_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            
            
            # standard_with_reasoning['top_id_is_A'] = standard_with_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            # standard_no_reasoning['top_id_is_A'] = standard_no_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            # standard_with_reasoning['top_id_is_other'] = standard_with_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            # standard_no_reasoning['top_id_is_other'] = standard_no_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            
            # newline_with_reasoning['top_id_is_A'] = newline_with_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            # newline_no_reasoning['top_id_is_A'] = newline_no_reasoning[id_colname].apply(lambda x: x[0] == token_A)
            # newline_with_reasoning['top_id_is_other'] = newline_with_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            # newline_no_reasoning['top_id_is_other'] = newline_no_reasoning[id_colname].apply(lambda x: x[0] != token_A and x[0] != token_B)
            
            # count number of token_A 
            results_temp['num_A_standard_prompt_with_reasoning'] = comparison_df['is_A_standard_with_reasoning'].value_counts().get(True, 0)
            results_temp['num_A_standard_prompt_no_reasoning'] = comparison_df['is_A_standard_no_reasoning'].value_counts().get(True, 0)
            results_temp['num_A_newline_prompt_with_reasoning'] = comparison_df['is_A_newline_with_reasoning'].value_counts().get(True, 0)
            results_temp['num_A_newline_prompt_no_reasoning'] = comparison_df['is_A_newline_no_reasoning'].value_counts().get(True, 0)
            
            
            # count number of other
            results_temp['num_other_standard_prompt_with_reasoning'] = comparison_df['is_other_standard_with_reasoning'].value_counts().get(True, 0)
            results_temp['num_other_standard_prompt_no_reasoning'] = comparison_df['is_other_standard_no_reasoning'].value_counts().get(True, 0)
            results_temp['num_other_newline_prompt_with_reasoning'] = comparison_df['is_other_newline_with_reasoning'].value_counts().get(True, 0)
            results_temp['num_other_newline_prompt_no_reasoning'] = comparison_df['is_other_newline_no_reasoning'].value_counts().get(True, 0)
        
            
            # num diff in top_id_is_A (between with and without reasoning)
            other_comparisons = pd.DataFrame()
            other_comparisons['standard_flipped'] = comparison_df['is_A_standard_with_reasoning'] != comparison_df['is_A_standard_no_reasoning']
            other_comparisons['newline_flipped'] = comparison_df['is_A_newline_with_reasoning'] != comparison_df['is_A_newline_no_reasoning']
            # other_comparisons['standard_flipped'] = standard_with_reasoning['top_id_is_A'] != standard_no_reasoning['top_id_is_A']
            # other_comparisons['newline_flipped'] = newline_with_reasoning['top_id_is_A'] != newline_no_reasoning['top_id_is_A']
            
            # num diff in top_id_is_A (between standard and newline)
            other_comparisons['prompt_var_flip_with_reasoning'] = comparison_df['is_A_standard_with_reasoning'] != comparison_df['is_A_newline_with_reasoning']
            other_comparisons['prompt_var_flip_no_reasoning'] = comparison_df['is_A_standard_no_reasoning'] != comparison_df['is_A_newline_no_reasoning']
            other_comparisons['reasoning_cause_flip_standard_prompt'] = comparison_df['is_A_standard_with_reasoning'] != comparison_df['is_A_standard_no_reasoning']
            other_comparisons['reasoning_cause_flip_newline_prompt'] = comparison_df['is_A_newline_with_reasoning'] != comparison_df['is_A_newline_no_reasoning']
            # other_comparisons['prompt_var_flip_with_reasoning'] = standard_with_reasoning['top_id_is_A'] != newline_with_reasoning['top_id_is_A']
            # other_comparisons['prompt_var_flip_no_reasoning'] = standard_no_reasoning['top_id_is_A'] != newline_no_reasoning['top_id_is_A']
            # other_comparisons['reasoning_cause_flip_standard_prompt'] = standard_with_reasoning['top_id_is_A'] != standard_no_reasoning['top_id_is_A']
            # other_comparisons['reasoning_cause_flip_newline_prompt'] = newline_with_reasoning['top_id_is_A'] != newline_no_reasoning['top_id_is_A']
            
            other_comparisons['reasoning_changed_prompt_var_flip'] = other_comparisons['standard_flipped'] != other_comparisons['newline_flipped']
            
            # add values to results df
            results_temp['num_prompt_var_cause_flip_with_reasoning'] = other_comparisons['prompt_var_flip_with_reasoning'].value_counts().get(True, 0)
            results_temp['num_prompt_var_cause_flip_no_reasoning'] = other_comparisons['prompt_var_flip_no_reasoning'].value_counts().get(True, 0)
            results_temp['num_reasoning_cause_flip_standard_prompt'] = other_comparisons['reasoning_cause_flip_standard_prompt'].value_counts().get(True, 0)
            results_temp['num_reasoning_cause_flip_newline_prompt'] = other_comparisons['reasoning_cause_flip_newline_prompt'].value_counts().get(True, 0)
            results_temp['num_reasoning_changed_prompt_var_flip'] = other_comparisons['reasoning_changed_prompt_var_flip'].value_counts().get(True, 0)
            
            # cat to results_full
            # print(results_temp.head())
            # print(results_full.head())
            row_df = pd.DataFrame([results_temp])

            results_full = pd.concat([results_full, row_df], ignore_index=True)
            
            # print(results_temp.head())
            # print(other_comparisons.head())
            # print(comparison_df.head())

print(results_full.head())
                
                
                
                
        

token A: 319, token B: 350


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


token A: 362, token B: 426
             model   dataset   run num_A_standard_prompt_with_reasoning  \
0      llama2-chat  harmless  run3                                    3   
1      llama2-chat  harmless  run4                                    3   
2      llama2-chat  dilemmas  run3                                   22   
3      llama2-chat  dilemmas  run4                                   22   
4  llama3-instruct  harmless  run3                                   53   

  num_A_newline_prompt_with_reasoning num_A_standard_prompt_no_reasoning  \
0                                   3                                  3   
1                                   3                                  3   
2                                  17                                 14   
3                                  17                                 14   
4                                  54                                 41   

  num_A_newline_prompt_no_reasoning num_other_standard_prompt_wit

In [32]:
# write results_full to jsonl
results_full.to_json('results_full.jsonl', orient='records', lines=True)
# write to csv
results_full.to_csv('results_full.csv', index=False)

In [33]:
def get_results(model, dataset, prompt_var):
    scores_without_reasoning_path = f'{results_path}/{model}/{dataset}/scores_without_reasoning_{prompt_var}.jsonl'
    scores_with_reasoning_path = f'{results_path}/{model}/{dataset}/scores_with_reasoning_{prompt_var}.jsonl'
    scores_without_reasoning = pd.read_json(scores_without_reasoning_path, orient='records', lines=True)
    scores_with_reasoning = pd.read_json(scores_with_reasoning_path, orient='records', lines=True)
    

In [34]:
token_A = tokenizer(' A', return_tensors='pt')['input_ids'][0]
token_B = tokenizer(' B', return_tensors='pt')['input_ids'][0]
print(token_A, token_B)

tensor([128000,    362]) tensor([128000,    426])


In [35]:
# tokenizer decode 426
B_tok_llama3 = tokenizer.decode(426)
print(B_tok_llama3[1])

# 362
A_tok_llama3 = tokenizer.decode(362)
print(len(A_tok_llama3))

B
2


In [36]:
# tokenizer for llama2
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

In [37]:
unexpected_token_ids = [29889, 29892, 4345, 7228, 29871]
print([tokenizer.decode(token_id) for token_id in unexpected_token_ids])

['.', ',', 'MS', 'PA', '']
